# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:31<00:00, 30.24s/it]


In [4]:
len(deals)

47

In [5]:
deals[44].describe()

"Title: Walmart Can't Miss Clearance: Up to 70% off + free shipping w/ $35\nDetails: Find clearance savings on clothing, home improvement, furniture, sports and outdoors, automotive, and more. Shipping adds $7, or get free shipping over $35. Shop Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Walmart-Cant-Miss-Clearance-Up-to-70-off-free-shipping-w-35/21765147.html?iref=rss-c196"

In [7]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [8]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Lax Gadgets 9-Outlet USB Swivel Surge Protector 2-Pack for $25 + free shipping
Details: You'd pay $7 more for two of these elsewhere today. This pack will ship for free from HSN, too. Buy Now at HSN
Features: 9 power outlets 2 USB ports 90-degree swivel in either direction Model: 889-489
URL: https://www.dealnews.com/products/Lax-Gadgets/Lax-Gadgets-9-Outlet-USB-Swi

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [13]:
result.deals[1]

Deal(product_description='The Oukitel P5000 Portable Power Station is a robust unit equipped with an impressive 5,120Wh capacity and a powerful 2200W AC pure sine wave inverter. It features lightning-fast AC input of 1800W and an uninterruptible power supply for emergencies. With the ability to support solar input up to 1000W, this power station is ideal for outdoor adventures or as a backup power source at home.', price=1289.0, url='https://www.dealnews.com/Oukitel-P5000-5-120-Wh-2-200-W-Portable-Power-Station-for-1289-free-shipping/21767464.html?iref=rss-c142')

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
result

DealSelection(deals=[Deal(product_description='The Lax Gadgets 9-Outlet USB Swivel Surge Protector is a versatile power management solution that features 9 power outlets and 2 USB ports, ensuring ample charging options for all your devices. Its 90-degree swivel design allows for easy placement in tight spaces, ensuring that all your plugs fit without obstruction. Ideal for home or office use, this surge protector protects your devices against power surges, making it a reliable choice for both everyday and sensitive electronics.', price=25.0, url='https://www.dealnews.com/products/Lax-Gadgets/Lax-Gadgets-9-Outlet-USB-Swivel-Surge-Protector-2-Pack/493789.html?iref=rss-c142'), Deal(product_description='The Oukitel P5000 Portable Power Station is a robust energy solution with a massive capacity of 5120Wh and a powerful 2200W AC Pure Sine Wave Inverter. It features a lightning-fast 1800W AC input for quick recharges and provides a reliable power supply during emergencies. Additionally, it s